In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile5 = pd.read_pickle("FFF_Train_Decile5.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile5.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile5, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 5')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 9s 3ms/step - loss: 0.4262 - acc: 0.8223 - val_loss: 0.3816 - val_acc: 0.8455
Epoch 36/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4208 - acc: 0.8308 - val_loss: 0.3803 - val_acc: 0.8390
Epoch 37/75
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4161 - acc: 0.8245 - val_loss: 0.3786 - val_acc: 0.8495
Epoch 38/75
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4268 - acc: 0.8259 - val_loss: 0.3784 - val_acc: 0.8442
Epoch 39/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4110 - acc: 0.8357 - val_loss: 0.3766 - val_acc: 0.8469
Epoch 40/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4182 - acc: 0.8236 - val_loss: 0.3746 - val_acc: 0.8521
Epoch 41/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4157 - acc: 0.8262 - val_loss: 0.3724 - val_acc: 0.8560
Epoch 42/75
3055/3055 [==============================] - 8s 3ms/ste

3055/3055 [==============================] - 9s 3ms/step - loss: 0.3594 - acc: 0.8462 - val_loss: 0.3282 - val_acc: 0.8639
Epoch 20/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3592 - acc: 0.8488 - val_loss: 0.3272 - val_acc: 0.8639
Epoch 21/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3622 - acc: 0.8475 - val_loss: 0.3284 - val_acc: 0.8639
Epoch 22/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3611 - acc: 0.8432 - val_loss: 0.3274 - val_acc: 0.8639
Epoch 23/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3611 - acc: 0.8517 - val_loss: 0.3269 - val_acc: 0.8665
Epoch 24/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3588 - acc: 0.8491 - val_loss: 0.3276 - val_acc: 0.8639
Epoch 25/75
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3582 - acc: 0.8455 - val_loss: 0.3273 - val_acc: 0.8652

Epoch 00025: ReduceLROnPlateau reducing learning rate to 1.0000000

3055/3055 [==============================] - 9s 3ms/step - loss: 0.4622 - acc: 0.7977 - val_loss: 0.4135 - val_acc: 0.8154
Epoch 4/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4590 - acc: 0.7967 - val_loss: 0.4192 - val_acc: 0.8233
Epoch 5/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4497 - acc: 0.8036 - val_loss: 0.4189 - val_acc: 0.8181
Epoch 6/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4392 - acc: 0.8111 - val_loss: 0.4020 - val_acc: 0.8285
Epoch 7/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4423 - acc: 0.7974 - val_loss: 0.4021 - val_acc: 0.8233
Epoch 8/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4384 - acc: 0.8115 - val_loss: 0.4011 - val_acc: 0.8272
Epoch 9/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4363 - acc: 0.8124 - val_loss: 0.3882 - val_acc: 0.8285
Epoch 10/100
3055/3055 [==============================] - 9s 3ms/st

3055/3055 [==============================] - 9s 3ms/step - loss: 0.3583 - acc: 0.8511 - val_loss: 0.3280 - val_acc: 0.8639
Epoch 63/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3513 - acc: 0.8537 - val_loss: 0.3276 - val_acc: 0.8613
Epoch 64/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3568 - acc: 0.8520 - val_loss: 0.3274 - val_acc: 0.8639
Epoch 65/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3569 - acc: 0.8530 - val_loss: 0.3273 - val_acc: 0.8652
Epoch 66/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3562 - acc: 0.8556 - val_loss: 0.3275 - val_acc: 0.8626
Epoch 67/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3542 - acc: 0.8524 - val_loss: 0.3276 - val_acc: 0.8613
Epoch 68/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3544 - acc: 0.8527 - val_loss: 0.3277 - val_acc: 0.8626
Epoch 69/100
3055/3055 [==============================] - 9s 

...

3055/3055 [==============================] - 8s 3ms/step - loss: 0.4209 - acc: 0.8350 - val_loss: 0.3760 - val_acc: 0.8469
Epoch 27/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4146 - acc: 0.8347 - val_loss: 0.3718 - val_acc: 0.8521
Epoch 28/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4141 - acc: 0.8429 - val_loss: 0.3706 - val_acc: 0.8534
Epoch 29/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4080 - acc: 0.8429 - val_loss: 0.3669 - val_acc: 0.8521
Epoch 30/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.4089 - acc: 0.8416 - val_loss: 0.3640 - val_acc: 0.8495
Epoch 31/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.4073 - acc: 0.8412 - val_loss: 0.3621 - val_acc: 0.8547
Epoch 32/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3974 - acc: 0.8406 - val_loss: 0.3608 - val_acc: 0.8573
Epoch 33/125
3055/3055 [==============================] - 9s 

Epoch 86/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3688 - acc: 0.8468 - val_loss: 0.3366 - val_acc: 0.8626
Epoch 87/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3726 - acc: 0.8491 - val_loss: 0.3369 - val_acc: 0.8626
Epoch 88/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.3665 - acc: 0.8491 - val_loss: 0.3372 - val_acc: 0.8626
Epoch 89/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3645 - acc: 0.8511 - val_loss: 0.3372 - val_acc: 0.8613
Epoch 90/125
3055/3055 [==============================] - 8s 3ms/step - loss: 0.3709 - acc: 0.8534 - val_loss: 0.3367 - val_acc: 0.8613
Epoch 91/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3694 - acc: 0.8442 - val_loss: 0.3365 - val_acc: 0.8626
Epoch 92/125
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3811 - acc: 0.8419 - val_loss: 0.3366 - val_acc: 0.8626
Epoch 93/125
3055/3055 [========================

3055/3055 [==============================] - 9s 3ms/step - loss: 0.3627 - acc: 0.8471 - val_loss: 0.3361 - val_acc: 0.8613
Epoch 21/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3595 - acc: 0.8471 - val_loss: 0.3276 - val_acc: 0.8613
Epoch 22/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3613 - acc: 0.8458 - val_loss: 0.3221 - val_acc: 0.8678
Epoch 23/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3531 - acc: 0.8481 - val_loss: 0.3224 - val_acc: 0.8704

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 24/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3470 - acc: 0.8553 - val_loss: 0.3219 - val_acc: 0.8704
Epoch 25/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3457 - acc: 0.8494 - val_loss: 0.3221 - val_acc: 0.8704
Epoch 26/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3458 - acc: 0.8534 - val_loss: 0.32

3055/3055 [==============================] - 8s 3ms/step - loss: 0.3542 - acc: 0.8484 - val_loss: 0.3227 - val_acc: 0.8665
Epoch 79/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3490 - acc: 0.8501 - val_loss: 0.3227 - val_acc: 0.8678
Epoch 80/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3503 - acc: 0.8465 - val_loss: 0.3226 - val_acc: 0.8678
Epoch 81/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3478 - acc: 0.8511 - val_loss: 0.3225 - val_acc: 0.8691
Epoch 82/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3541 - acc: 0.8514 - val_loss: 0.3226 - val_acc: 0.8691
Epoch 83/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3473 - acc: 0.8530 - val_loss: 0.3228 - val_acc: 0.8652
Epoch 84/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.3489 - acc: 0.8501 - val_loss: 0.3227 - val_acc: 0.8665
Epoch 85/100
3055/3055 [==============================] - 9s 

3055/3055 [==============================] - 6s 2ms/step - loss: 0.3608 - acc: 0.8458 - val_loss: 0.3271 - val_acc: 0.8626
Epoch 39/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3627 - acc: 0.8465 - val_loss: 0.3243 - val_acc: 0.8639
Epoch 40/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3598 - acc: 0.8488 - val_loss: 0.3243 - val_acc: 0.8639
Epoch 41/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3552 - acc: 0.8520 - val_loss: 0.3306 - val_acc: 0.8613
Epoch 42/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3625 - acc: 0.8484 - val_loss: 0.3229 - val_acc: 0.8652
Epoch 43/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3618 - acc: 0.8478 - val_loss: 0.3311 - val_acc: 0.8639
Epoch 44/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3624 - acc: 0.8511 - val_loss: 0.3240 - val_acc: 0.8678
Epoch 45/50
3055/3055 [==============================] - 6s 2ms/ste

In [3]:
0.93484

0.93484